In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary
from sklearn.model_selection import train_test_split

from model_pytorch import EfficientNet
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections

In [2]:
# GPU Device
gpu_id = 0
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 0: True


# Arguments

In [3]:
data_dir = '/media/data2/dataset/GAN_ImageData/StarGAN_128/'

In [4]:
pretrained = ''
resume = ''

In [5]:
# Model
model_name = 'efficientnet-b0' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 200
start_epoch = 0
train_batch = 200
test_batch = 200
lr = 0.015
schedule = [20, 75, 125, 175]
momentum = 0.9
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/star/128/b0' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 4

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

best_acc = 0

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

# Dataset

In [7]:
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'validation')    
train_aug = transforms.Compose([
    transforms.RandomAffine(degrees=2, translate=(0.02, 0.02), scale=(0.98, 1.02), shear=2, fillcolor=(124,117,104)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.10), ratio=(0.3, 3.3), value=0, inplace=True),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(datasets.ImageFolder(val_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [8]:
model = EfficientNet.from_name(model_name, num_classes=num_classes,
                              override_params={'dropout_rate':0.3, 'drop_connect_rate':0.3})

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    model.load_state_dict(torch.load(pretrained)['state_dict'])

In [9]:
model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

    Total params: 4.01M


In [10]:
# summary(model, input_size=(3,64,64), device='cuda')

# Loss

In [11]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
# optimizer = optim.Adam(model.parameters(), weight_decay=1e-4)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=8, total_epoch=10, after_scheduler=scheduler_cosine)

In [12]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Train Acc.', 'Valid Acc.'])

# Train

In [13]:
def train(train_loader, model, criterion, optimizer, epoch, use_cuda):
    model.train()
    torch.set_grad_enabled(True)
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    end = time.time()
    
    bar = Bar('Processing', max=len(train_loader))
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:.4f} | top1: {top1: .4f} '.format(
                    batch=batch_idx + 1,
                    size=len(train_loader),
                    data=data_time.val,
                    bt=batch_time.val,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,
                    )
        bar.next()
        if batch_idx % 10 == 0:
            print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
                 batch=batch_idx+1, size=len(train_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [14]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()
    torch.set_grad_enabled(False)

    end = time.time()
    bar = Bar('Processing', max=len(val_loader))
    for batch_idx, (inputs, targets) in enumerate(val_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = torch.autograd.Variable(inputs, volatile=True), torch.autograd.Variable(targets)

        # compute output
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        # plot progress
        bar.suffix  = '({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | ETA: {eta:} | Loss: {loss:} | top1: {top1:}'.format(
                    batch=batch_idx + 1,
                    size=len(val_loader),
                    data=data_time.avg,
                    bt=batch_time.avg,
                    total=bar.elapsed_td,
                    eta=bar.eta_td,
                    loss=losses.avg,
                    top1=top1.avg,)
        bar.next()
    print('{batch}/{size} Data:{data:.3f} | Batch:{bt:.3f} | Total:{total:} | ETA:{eta:} | Loss:{loss:} | top1:{tp1:}'.format(
         batch=batch_idx+1, size=len(val_loader), data=data_time.val, bt=batch_time.val, total=bar.elapsed_td, eta=bar.eta_td, loss=losses.avg, tp1=top1.avg))
    bar.finish()
    return (losses.avg, top1.avg)

In [15]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [16]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    train_loss, train_acc = train(train_loader, model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc = test(val_loader, model, criterion, epoch, use_cuda)
    
    logger.append([state['lr'], train_loss, test_loss, train_acc, test_acc])
    scheduler_warmup.step()

    is_best = test_acc > best_acc
    best_acc = max(test_acc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)


Epoch: [1 | 200] LR: 0.015000
1/1373 Data:0.636 | Batch:4.529 | Total:0:00:04 | ETA:1:43:35 | Loss:0.6803147196769714 | top1:60.5
11/1373 Data:0.041 | Batch:0.503 | Total:0:00:09 | ETA:0:20:26 | Loss:0.7092232649976556 | top1:48.09090805053711
21/1373 Data:0.038 | Batch:0.497 | Total:0:00:14 | ETA:0:11:09 | Loss:0.7076750113850548 | top1:48.66666793823242
31/1373 Data:0.043 | Batch:0.497 | Total:0:00:19 | ETA:0:11:07 | Loss:0.7095572467773191 | top1:48.85483932495117
41/1373 Data:0.041 | Batch:0.491 | Total:0:00:24 | ETA:0:11:03 | Loss:0.710116643731187 | top1:49.34146499633789
51/1373 Data:0.042 | Batch:0.497 | Total:0:00:29 | ETA:0:10:56 | Loss:0.7081257373678917 | top1:49.57843017578125
61/1373 Data:0.041 | Batch:0.492 | Total:0:00:34 | ETA:0:10:51 | Loss:0.7075279247565348 | top1:49.6065559387207
71/1373 Data:0.041 | Batch:0.497 | Total:0:00:39 | ETA:0:10:45 | Loss:0.7075680351593125 | top1:49.640846252441406
81/1373 Data:0.041 | Batch:0.497 | Total:0:00:44 | ETA:0:10:39 | Loss:0.

721/1373 Data:0.037 | Batch:0.464 | Total:0:05:59 | ETA:0:05:14 | Loss:0.7023459710129091 | top1:50.02912902832031
731/1373 Data:0.039 | Batch:0.471 | Total:0:06:04 | ETA:0:05:07 | Loss:0.702332549887708 | top1:50.02394104003906
741/1373 Data:0.035 | Batch:0.479 | Total:0:06:09 | ETA:0:05:03 | Loss:0.7022961070019546 | top1:50.00135040283203
751/1373 Data:0.024 | Batch:0.239 | Total:0:06:12 | ETA:0:04:10 | Loss:0.7022542905870989 | top1:50.00931930541992
761/1373 Data:0.041 | Batch:0.493 | Total:0:06:17 | ETA:0:04:53 | Loss:0.7022137551050776 | top1:50.00459671020508
771/1373 Data:0.043 | Batch:0.501 | Total:0:06:22 | ETA:0:04:59 | Loss:0.7021721989573517 | top1:50.005836486816406
781/1373 Data:0.038 | Batch:0.497 | Total:0:06:27 | ETA:0:04:54 | Loss:0.7020748713257676 | top1:50.02112579345703
791/1373 Data:0.040 | Batch:0.494 | Total:0:06:32 | ETA:0:04:49 | Loss:0.7019799308740686 | top1:50.02465057373047
801/1373 Data:0.043 | Batch:0.500 | Total:0:06:36 | ETA:0:04:45 | Loss:0.7019116

/home/cutz/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


153/153 Data:0.002 | Batch:0.943 | Total:0:00:22 | ETA:0:00:00 | Loss:0.6931968054796421 | top1:50.203147888183594

Epoch: [2 | 200] LR: 0.025500
1/1373 Data:0.685 | Batch:1.155 | Total:0:00:01 | ETA:0:27:21 | Loss:0.6988340616226196 | top1:48.0
11/1373 Data:0.041 | Batch:0.499 | Total:0:00:06 | ETA:0:12:52 | Loss:0.694625735282898 | top1:51.0
21/1373 Data:0.041 | Batch:0.494 | Total:0:00:11 | ETA:0:11:11 | Loss:0.6961177701041812 | top1:50.380950927734375
31/1373 Data:0.043 | Batch:0.498 | Total:0:00:16 | ETA:0:11:05 | Loss:0.695628373853622 | top1:50.209678649902344
41/1373 Data:0.041 | Batch:0.480 | Total:0:00:21 | ETA:0:11:02 | Loss:0.6953858980318395 | top1:50.10975646972656
51/1373 Data:0.041 | Batch:0.494 | Total:0:00:25 | ETA:0:10:56 | Loss:0.6954991268176659 | top1:50.186275482177734
61/1373 Data:0.040 | Batch:0.499 | Total:0:00:30 | ETA:0:10:51 | Loss:0.6960679888725281 | top1:49.803279876708984
71/1373 Data:0.043 | Batch:0.500 | Total:0:00:35 | ETA:0:10:47 | Loss:0.696186142

711/1373 Data:0.043 | Batch:0.499 | Total:0:05:49 | ETA:0:05:28 | Loss:0.6962883598190822 | top1:49.88045120239258
721/1373 Data:0.043 | Batch:0.497 | Total:0:05:54 | ETA:0:05:25 | Loss:0.6962481942156978 | top1:49.89320373535156
731/1373 Data:0.041 | Batch:0.494 | Total:0:05:59 | ETA:0:05:19 | Loss:0.6962406323807347 | top1:49.890560150146484
741/1373 Data:0.041 | Batch:0.495 | Total:0:06:04 | ETA:0:05:15 | Loss:0.6962245212071015 | top1:49.87449645996094
751/1373 Data:0.041 | Batch:0.500 | Total:0:06:09 | ETA:0:05:11 | Loss:0.6961996999783776 | top1:49.86018753051758
761/1373 Data:0.041 | Batch:0.502 | Total:0:06:14 | ETA:0:05:05 | Loss:0.6961790021709637 | top1:49.8607063293457
771/1373 Data:0.041 | Batch:0.494 | Total:0:06:19 | ETA:0:04:59 | Loss:0.6961449890285151 | top1:49.85927200317383
781/1373 Data:0.043 | Batch:0.493 | Total:0:06:24 | ETA:0:04:54 | Loss:0.6961271067587576 | top1:49.85979461669922
791/1373 Data:0.044 | Batch:0.499 | Total:0:06:29 | ETA:0:04:49 | Loss:0.6960867

...

671/1373 Data:0.041 | Batch:0.495 | Total:0:05:30 | ETA:0:05:50 | Loss:0.03142488676181616 | top1:98.79508209228516
681/1373 Data:0.041 | Batch:0.499 | Total:0:05:35 | ETA:0:05:45 | Loss:0.03138954053282814 | top1:98.799560546875
691/1373 Data:0.043 | Batch:0.503 | Total:0:05:40 | ETA:0:05:40 | Loss:0.03149788617628172 | top1:98.79594421386719
701/1373 Data:0.041 | Batch:0.496 | Total:0:05:45 | ETA:0:05:34 | Loss:0.031504133706593544 | top1:98.79885864257812
711/1373 Data:0.041 | Batch:0.498 | Total:0:05:50 | ETA:0:05:30 | Loss:0.03149151240878918 | top1:98.80098724365234
721/1373 Data:0.043 | Batch:0.498 | Total:0:05:55 | ETA:0:05:25 | Loss:0.03153546237911197 | top1:98.80236053466797
731/1373 Data:0.041 | Batch:0.487 | Total:0:06:00 | ETA:0:05:20 | Loss:0.031527806383974226 | top1:98.80438232421875
741/1373 Data:0.043 | Batch:0.503 | Total:0:06:05 | ETA:0:05:15 | Loss:0.03158375002693903 | top1:98.80162048339844
751/1373 Data:0.041 | Batch:0.496 | Total:0:06:10 | ETA:0:05:09 | Loss:0

153/153 Data:0.002 | Batch:0.087 | Total:0:00:21 | ETA:0:00:00 | Loss:0.029431487406759008 | top1:98.95806884765625

Epoch: [199 | 200] LR: 0.000000
1/1373 Data:0.592 | Batch:1.071 | Total:0:00:01 | ETA:0:25:22 | Loss:0.0454372838139534 | top1:98.0
11/1373 Data:0.041 | Batch:0.498 | Total:0:00:06 | ETA:0:12:43 | Loss:0.028242145834321327 | top1:99.0
21/1373 Data:0.041 | Batch:0.498 | Total:0:00:11 | ETA:0:11:14 | Loss:0.03098366116838796 | top1:98.92857360839844
31/1373 Data:0.041 | Batch:0.495 | Total:0:00:16 | ETA:0:11:09 | Loss:0.031016698107123375 | top1:98.85484313964844
41/1373 Data:0.041 | Batch:0.501 | Total:0:00:21 | ETA:0:11:03 | Loss:0.03191943657470912 | top1:98.79268646240234
51/1373 Data:0.039 | Batch:0.500 | Total:0:00:26 | ETA:0:10:58 | Loss:0.030959612105553057 | top1:98.84313201904297
61/1373 Data:0.041 | Batch:0.494 | Total:0:00:30 | ETA:0:10:54 | Loss:0.03096425745514084 | top1:98.83606719970703
71/1373 Data:0.041 | Batch:0.495 | Total:0:00:35 | ETA:0:10:48 | Loss:0

701/1373 Data:0.041 | Batch:0.494 | Total:0:05:49 | ETA:0:05:35 | Loss:0.03194570833931381 | top1:98.77960205078125
711/1373 Data:0.041 | Batch:0.495 | Total:0:05:54 | ETA:0:05:30 | Loss:0.03187799535471679 | top1:98.78059387207031
721/1373 Data:0.043 | Batch:0.502 | Total:0:05:59 | ETA:0:05:25 | Loss:0.031890306884006965 | top1:98.77947235107422
731/1373 Data:0.040 | Batch:0.503 | Total:0:06:04 | ETA:0:05:21 | Loss:0.031939810754733844 | top1:98.77770233154297
741/1373 Data:0.041 | Batch:0.499 | Total:0:06:09 | ETA:0:05:16 | Loss:0.031970094386035736 | top1:98.77800750732422
751/1373 Data:0.041 | Batch:0.495 | Total:0:06:14 | ETA:0:05:10 | Loss:0.03179764900959225 | top1:98.78694915771484
761/1373 Data:0.041 | Batch:0.495 | Total:0:06:19 | ETA:0:05:05 | Loss:0.031782348028967906 | top1:98.78974914550781
771/1373 Data:0.036 | Batch:0.496 | Total:0:06:24 | ETA:0:05:00 | Loss:0.03178757912722137 | top1:98.78728485107422
781/1373 Data:0.041 | Batch:0.500 | Total:0:06:29 | ETA:0:04:55 | Lo

21/1373 Data:0.041 | Batch:0.500 | Total:0:00:11 | ETA:0:11:14 | Loss:0.02913396078206244 | top1:98.92857360839844
31/1373 Data:0.041 | Batch:0.498 | Total:0:00:16 | ETA:0:11:10 | Loss:0.030193371818430962 | top1:98.8709716796875
41/1373 Data:0.041 | Batch:0.496 | Total:0:00:21 | ETA:0:11:04 | Loss:0.03053715840981501 | top1:98.91463470458984
51/1373 Data:0.043 | Batch:0.499 | Total:0:00:26 | ETA:0:10:59 | Loss:0.031237746114093884 | top1:98.9313735961914
61/1373 Data:0.044 | Batch:0.503 | Total:0:00:31 | ETA:0:10:54 | Loss:0.031325892575818005 | top1:98.92623138427734
71/1373 Data:0.043 | Batch:0.503 | Total:0:00:35 | ETA:0:10:48 | Loss:0.0304699216536443 | top1:98.9507064819336
81/1373 Data:0.041 | Batch:0.494 | Total:0:00:40 | ETA:0:10:43 | Loss:0.031430196695397664 | top1:98.86419677734375
91/1373 Data:0.041 | Batch:0.495 | Total:0:00:45 | ETA:0:10:38 | Loss:0.03206911395498357 | top1:98.80769348144531
101/1373 Data:0.041 | Batch:0.492 | Total:0:00:50 | ETA:0:10:34 | Loss:0.0321543

731/1373 Data:0.041 | Batch:0.502 | Total:0:06:00 | ETA:0:05:20 | Loss:0.03195053806959074 | top1:98.77291870117188
741/1373 Data:0.039 | Batch:0.498 | Total:0:06:05 | ETA:0:05:15 | Loss:0.031862264716502546 | top1:98.77732849121094
751/1373 Data:0.041 | Batch:0.495 | Total:0:06:10 | ETA:0:05:10 | Loss:0.031913134784061964 | top1:98.77429962158203
761/1373 Data:0.041 | Batch:0.495 | Total:0:06:15 | ETA:0:05:05 | Loss:0.031905520213559836 | top1:98.7746353149414
771/1373 Data:0.041 | Batch:0.496 | Total:0:06:20 | ETA:0:05:00 | Loss:0.031932318292621005 | top1:98.77496337890625
781/1373 Data:0.035 | Batch:0.495 | Total:0:06:25 | ETA:0:04:55 | Loss:0.03184671811675999 | top1:98.77720642089844
791/1373 Data:0.041 | Batch:0.500 | Total:0:06:30 | ETA:0:04:51 | Loss:0.03191683171364077 | top1:98.77559661865234
801/1373 Data:0.041 | Batch:0.498 | Total:0:06:35 | ETA:0:04:45 | Loss:0.03184906704660584 | top1:98.77840423583984
811/1373 Data:0.041 | Batch:0.498 | Total:0:06:40 | ETA:0:04:40 | Los